In [ ]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve, auc, accuracy_score
from sklearn import linear_model
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

#Esta notebook es un primer boceto de lo que habría que hacer.
Ojo con el nombre de las series que se me pudo haber escapado algo de los ejemplos.

Tener en cuenta la creación de nuestras X e y.

En el modelo Naive Bayes Multinomial la mayoría de los ejemplos es con clasificador de texto.

Ojo con la estandarización de aplicarlo al set de entreamiento y llevar esa transformación al de testeo.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('df_ecobicis_final.csv')

In [ ]:
df.sample(10)

In [ ]:
df.columns

In [ ]:
#Revisando valores nulos
df.isnull().sum()

In [ ]:
# visualizamos la matriz de correlación en Seaborn usando a heatmap

sns.heatmap(df.corr(), vmin=-1, vmax=1, center=0, cmap="YlGnBu");

In [ ]:
#Generando variables dummies
dummies_dia = pd.get_dummies(df.dia, prefix=None, prefix_sep='_', drop_first=True)
dummies_sexo=pd.get_dummies(df.bici_sexo, prefix=None, prefix_sep='_', drop_first=True)
dummies_origen=pd.get_dummies(df.bici_nombre_estacion_origen, prefix=None, prefix_sep='_', drop_first=True)
dummies_comuna=pd.get_dummies(df.comuna_origen, prefix='comuna', prefix_sep='_', drop_first=True)
dummies_estacion_agno=pd.get_dummies(df.estacion_agno, prefix=None, prefix_sep='_', drop_first=True)
dummies_rango=pd.get_dummies(df.rango_horario, prefix=None, prefix_sep='_', drop_first=True)


In [ ]:
#Concatenamos los df de las dummies
df=pd.concat([df,dummies_sexo,dummies_dia,dummies_comuna,dummies_rango,dummies_origen,dummies_estacion_agno], axis=1)

In [ ]:
df.head()

In [ ]:
#Dropear las features redundantes o que no necesitamos
df = df.drop(['barrio_destino'], axis=1)
df = df.drop(['barrio_origen'], axis=1)
df = df.drop(['bici_nombre_estacion_destino'], axis=1)
df = df.drop(['bici_nombre_estacion_origen'], axis=1)
df = df.drop(['dia'], axis=1)
df = df.drop(['Unnamed: 0'], axis=1)
df=df.drop(['bici_id_usuario'], axis=1)
df=df.drop(['bici_Fecha_hora_retiro'], axis=1)
df=df.drop(['bici_estacion_origen'], axis=1)
df=df.drop(['bici_estacion_destino'], axis=1)
df=df.drop(['fecha'], axis=1)
df=df.drop(['dia_semana'], axis=1)
df=df.drop(['bici_sexo'], axis=1)
df=df.drop(['comuna_destino'], axis=1)
df=df.drop(['hora'], axis=1)
df=df.drop(['estacion_agno'], axis=1)
df=df.drop(['rango_horario'], axis=1)
df=df.drop(['bici_tiempo_uso'], axis=1)



df.head()

In [ ]:
# Para Entrenamiento
df_bajo_E = df.loc[(df['uso_categorico']=='BAJO') & (df['uso_categorico'].index % 2 == 0),:].sample(25000, random_state= 5)
df_medio_E = df.loc[(df['uso_categorico']=='MEDIO') & (df['uso_categorico'].index % 2 == 0),:].sample(25000, random_state= 15)
df_alto_E = df.loc[(df['uso_categorico']=='ALTO') & (df['uso_categorico'].index % 2 == 0),:].sample(25000, random_state= 25)
df_excedido_E = df.loc[(df['uso_categorico']=='EXCEDIDO') & (df['uso_categorico'].index % 2 == 0),:].sample(25000, random_state= 35)

In [ ]:
df_balance = pd.concat([df_bajo_E, df_medio_E, df_alto_E,  df_excedido_E])

In [ ]:
y_train=df_balance['uso_categorico']

In [ ]:
X_train=df_balance.drop(['uso_categorico'], axis=1)

In [ ]:
df_test = df.loc[(df['uso_categorico'].index % 2 == 1),:].sample(200000, random_state= 5)

In [ ]:
y_test=df_test['uso_categorico']

In [ ]:
X_test=df_test.drop(['uso_categorico'], axis=1)

## Clasificación (SVN)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=12)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC


In [ ]:
svc_C=[1, 5, 10, 50]

In [ ]:
svc_gamma = [0.0001, 0.0005, 0.01, 0.05]

In [ ]:
svc = SVC(kernel='rbf')
grid = GridSearchCV(svc, param_grid={'C': svc_C, 'gamma': svc_gamma}, scoring='neg_log_loss')
grid.fit(X_train, y_train)

In [ ]:
grid.cv_results_.keys()

In [ ]:
pd.DataFrame(grid.cv_results_).columns

In [ ]:
pd.DataFrame(grid.cv_results_)

In [ ]:
grid.best_estimator_, grid.best_score_, grid.best_params_

In [ ]:
#en C y gamma van los valores que te de el greedserch
model_svc = SVC(kernel='rbf', C=c, gamma = g)

In [ ]:
model_svc.fit(X_train_pca,ytrain)